# SABER Temperature Trend Analysis (2D, Silver Layer)

This notebook processes **bronze-level aggregated SABER temperature data** and applies multiple 
statistical methods to detect trends across altitudes in the Mesosphere-Lower Thermosphere (MLT).

**Pipeline stages:**
1. Setup paths and imports.
2. Load custom functions.
3. Load bronze-level input data.
4. Run trend analyses:
   - Mann-Kendall test
   - Sequential Mann-Kendall test
   - Sen’s slope estimator
   - Innovative Trend Analysis (ITA)
5. Aggregate results.
6. Save outputs to the silver layer (CSV + joblib).


---
**Input:**  
- `bronze_saber_2d_tempoaral_agg.csv`

**Output:**  
- `silver_smk_2d.joblib`
- `silver_smk_2d_count.csv`  
- `silver_mk_2d.csv`  
- `silver_sen_slope_2d.csv`  
- `silver_ita_2d.csv`  
---

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pymannkendall as mk
from scipy.stats import linregress
from scipy.stats import theilslopes
import datetime as dt
from scipy.stats import norm
import joblib
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Current working directory (notebook folder)
notebook_path = Path.cwd()  # MLT/code/
base_path = notebook_path.parent.parent  # Go up to MLT/

functions_path = notebook_path.parent / "000_Functions.ipynb"
data_path = base_path / "data" / "bronze"
silver_path = base_path / "data" / "silver"

In [3]:
# Load custom functions
%run "{functions_path.as_posix()}"

In [4]:
 # Read and concatenate all CSVs
df = pd.read_csv(data_path / "bronze_saber_2d_tempoaral_agg.csv")

In [5]:
df['year_month'] = pd.to_datetime(df['year_month'], errors='coerce')
df['year_month'] = df['year_month'].dt.to_period('M')

## Run Trend Analyses

### Mann-Kendall Test by Altitude
**Output:** silver_mk_2d.csv

In [7]:
mk_2d = run_mk_tests_by_altitude(df, mk)

### Sequential Mann-Kendall Test by Altitude
**Output:** silver_smk_2d.csv and silver_smk_2d_count.csv

In [8]:
smk_2d = run_sequential_mk_by_altitude(df)

In [9]:
smk_2d_count = analyze_trends_count(smk_2d)

### Innovative Trend Analysis (ITA)
**Output:** silver_ita_2d.csv

In [10]:
ita_2d = ITA_alt(df)

### Sen’s Slope Estimation
**Output:** silver_sen_slope_2d.csv

In [11]:
sen_slope_2d = sen_slope_by_altitude(df)

## 6. Save Silver-layer Outputs

In [12]:
# Ensure silver folder exists
silver_path.mkdir(parents=True, exist_ok=True)

# Define tables to save
tables = {
    "silver_smk_2d_count": smk_2d_count,
    "silver_mk_2d": mk_2d,
    "silver_sen_slope_2d": sen_slope_2d,
    "silver_ita_2d": ita_2d,
}

# Save tables to CSV
for table_name, table in tables.items():
    try:
        file_path = silver_path / f"{table_name}.csv"
        table.to_csv(file_path, index=False)
        print(f"[SUCCESS] {table_name}.csv saved")
    except Exception as e:
        print(f"[ERROR] Could not save {table_name}. Reason: {e}")

print("All CSVs saved successfully.")

[SUCCESS] silver_smk_2d_count.csv saved
[SUCCESS] silver_mk_2d.csv saved
[SUCCESS] silver_sen_slope_2d.csv saved
[SUCCESS] silver_ita_2d.csv saved
All CSVs saved successfully.


In [13]:
# Save
joblib.dump(smk_2d, silver_path / "silver_smk_2d.joblib")

['C:\\Users\\mvana\\Documents\\MSc\\MLT\\data\\silver\\silver_smk_2d.joblib']